
# Load data from Log Analytics to Delta Lake

This notebook shows you how to import data from Log Analytics workspace into a Delta Lake table using Python.

# Prerequisites

In [ ]:
#!pip install azure-monitor-query

In [ ]:
import os
from datetime import datetime, timedelta, timezone

from azure.monitor.query import LogsQueryClient, LogsQueryStatus
from azure.identity import ClientSecretCredential
from azure.core.exceptions import HttpResponseError

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import pandas as pd

In [ ]:
## Variables
db = "deltadb"
table_format = "delta"
table_name = "process_execution"

In [ ]:
## Creates database if not exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")
spark.sql("SET spark.databricks.delta.formatCheck.enabled = false")
spark.sql("SET spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite = true")

In [ ]:
# Creating a SparkSession
spark = SparkSession.builder.appName("DataFrame").getOrCreate()


## Step 1: Connection information

First define some variables to programmatically create these connections.

Replace all the variables in angle brackets `<>` below with the corresponding information.

In [ ]:
## Variables
# The client_secret should be kept in the Databricks Secrets/Key Vault as security best practice.
tenant_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
workspace_id = 'www-xxx-yyy-zzz'
client_id = 'yyyyyyyy-yyyy-yyyy-yyyy-yyyyyyyyyyyy'
# client_secret should be stored in Databricks Secrets/Key Vault
# client_secret = dbutils.secrets.get(scope = 'YOUR_SCOPE_HERE', key = 'YOUR_KEY_HERE')
client_secret = 'zzzzzz'

In [ ]:
## Creates credential object and instantiate logs_client
credential = ClientSecretCredential(
       tenant_id=tenant_id,
       client_id=client_id,
       client_secret=client_secret,
   )

logs_client = LogsQueryClient(credential)


## Step 2: Reading the data

Now that you've specified the file metadata, you can create a DataFrame. Use an *option* to infer the data schema from the file. You can also explicitly set this to a particular schema if you have one already.

First, create a DataFrame in Python, referencing the variables defined above.

In [ ]:
## Query parameters
query = """SecurityEvent
| where EventID == 4688
| where Account != @'-\-'
| project TimeGenerated, Account, AccountType, Computer, Process, CommandLine"""

end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days = 90)

In [ ]:
## Retrieve data
try:
    response = logs_client.query_workspace(
        workspace_id=workspace_id,
        query=query,
        timespan=(start_time, end_time)
        )
    if response.status == LogsQueryStatus.PARTIAL:
        error = response.partial_error
        data = response.partial_data
        print(error)
    elif response.status == LogsQueryStatus.SUCCESS:
        data = response.tables
    for table in data:
        # Converting the table data to a Spark DataFrame
        pandas_df = pd.DataFrame(data=table.rows, columns=table.columns)
        df=spark.createDataFrame(pandas_df) 
except HttpResponseError as err:
    print("something fatal happened")
    print(err)

In [ ]:
## DF Schema
df.describe().show()


## Step 3: Create a Delta table

The DataFrame defined and displayed above is a temporary connection to the remote database.

To ensure that this data can be accessed by relevant users throughout your workspace, save it as a Delta Lake table using the code below.

In [ ]:
df.write.mode("overwrite").saveAsTable(table_name)


This table will persist across cluster sessions, notebooks, and personas throughout your organization.

The code below demonstrates querying this data with Python and SQL.

In [ ]:
## Display data table
display(spark.table(table_name))